# <center>TSP: Travelling Salesman Problem</center>

### The traveling salesman problem (TSP) is a well-known combinatorial optimization problem that seeks to find the shortest possible route that visits each of a set of cities and returns to the starting city.

In [1]:
import itertools

def tsp(cities):
    # Create a list of all possible city permutations
    routes = list(itertools.permutations(cities))

    # Compute the distance of each route and select the shortest one
    shortest_route = None
    shortest_distance = float('inf')

    for route in routes:
        distance = 0
        for i in range(len(route) - 1):
            distance += get_distance(route[i], route[i+1])
        if distance < shortest_distance:
            shortest_distance = distance
            shortest_route = route

    return shortest_route, shortest_distance

def get_distance(city1, city2):
    # Compute the Euclidean distance between two cities
    return ((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2) ** 0.5

# Example usage
cities = [(0, 0), (1, 2), (3, 1), (2, 3)]
shortest_route, shortest_distance = tsp(cities)
print("Shortest route: ", shortest_route)
print("Shortest distance: ", shortest_distance)


Shortest route:  ((0, 0), (1, 2), (2, 3), (3, 1))
Shortest distance:  5.8863495173726745


**The tsp function takes a list of cities (represented as (x, y) coordinates) as input and returns the shortest route and distance.** 

* The **get_distance** function computes the Euclidean distance between two cities. 

* The implementation uses the **brute force approach** to generate all possible city permutations and select the one with the shortest distance. 

* This approach is **inefficient for large numbers of cities**, but it guarantees an optimal solution.

##### For large numbers of cities, the brute force approach used in the previous implementation becomes impractical due to the large number of possible routes to consider. In such cases, heuristic and approximation algorithms can be used to find an approximate solution that is close to the optimal solution in a reasonable amount of time

**One commonly used algorithm for the TSP is the nearest neighbor algorithm, which works as follows:**

* Start at an arbitrary city.
* Find the nearest unvisited city and add it to the tour.
* Repeat step 2 until all cities have been visited.
* Return to the starting city to complete the tour.

This algorithm produces a valid tour but does not guarantee an optimal solution. However, it is much faster than the brute force approach and can be used to find a good solution for large numbers of cities.

### Nearest Neighbor:

In [4]:
import math

def tsp_nn(cities):
    '''
    The tsp_nn function takes a list of cities as input and returns the shortest route and distance
    using the nearest neighbor algorithm.
    
    The implementation starts at the first city and iteratively selects the nearest unvisited city 
    until all cities have been visited.
    '''
    n = len(cities)
    visited = [False] * n
    route = [0] * n

    visited[0] = True
    route[0] = 0

    for i in range(1, n):
        # Find the nearest unvisited city
        nearest_dist = float('inf')
        nearest_city = None
        for j in range(n):
            if not visited[j]:
                dist = get_distance(cities[route[i-1]], cities[j])
                if dist < nearest_dist:
                    nearest_dist = dist
                    nearest_city = j
        route[i] = nearest_city
        visited[nearest_city] = True

    # Compute the distance of the route and return it
    distance = 0
    for i in range(n):
        distance += get_distance(cities[route[i]], cities[route[(i+1) % n]])
    return route, distance

def get_distance(city1, city2):
    '''
    The get_distance function computes the Euclidean distance between two cities.
    '''
    # Compute the Euclidean distance between two cities
    return math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Example usage
cities = [(0, 0), (1, 2), (3, 1), (2, 3)]
route, distance = tsp_nn(cities)
print("Route: ", route)
print("Distance: ", distance)


Route:  [0, 1, 3, 2]
Distance:  9.048627177541054


##### Another approach is to use metaheuristic algorithms such as simulated annealing, genetic algorithms, or ant colony optimization. These algorithms can be used to search for good solutions in large solution spaces, but they may not always find the optimal solution. These algorithms can be effective when the goal is to find a good solution quickly, rather than an optimal solution

<I>In summary, for large numbers of cities, heuristic and approximation algorithms can be used to find good solutions in a reasonable amount of time, while metaheuristic algorithms can be used to search for good solutions in large solution spaces.</I>

## Simulated Annealing:

Simulated annealing is a metaheuristic optimization algorithm that is inspired by the process of annealing in metallurgy. The algorithm starts with an initial solution and iteratively perturbs the solution and evaluates the resulting change in the objective function. The algorithm accepts the new solution if it is better than the previous solution and accepts worse solutions with a certain probability that decreases over time. This allows the algorithm to escape from local optima and explore the solution space more thoroughly.

In [5]:
import math
import random

def tsp_sa(cities, T=1.0, alpha=0.99, stopping_temp=1e-8, stopping_iter=100000):
    """
    Traveling Salesman Problem using Simulated Annealing.

    Parameters:
        cities : list
            List of city coordinates (x, y).
        T : float
            Initial temperature.
        alpha : float
            Cooling factor.
        stopping_temp : float
            Temperature to stop iterations.
        stopping_iter : int
            Maximum number of iterations.

    Returns:
        A tuple containing the shortest route and the distance of the route.
    """
    n = len(cities)
    route = list(range(n))
    random.shuffle(route)
    distance = compute_distance(route, cities)
    i = 0

    while T > stopping_temp and i < stopping_iter:
        # Choose two random cities to swap
        a, b = random.sample(range(n), 2)

        # Swap them and compute the new distance
        new_route = route.copy()
        new_route[a], new_route[b] = new_route[b], new_route[a]
        new_distance = compute_distance(new_route, cities)

        # Compute the acceptance probability
        delta = new_distance - distance
        acceptance_prob = math.exp(-delta / T)

        # Accept or reject the new solution
        if delta < 0 or random.random() < acceptance_prob:
            route = new_route
            distance = new_distance

        # Decrease the temperature and increase the iteration counter
        T *= alpha
        i += 1

    return route, distance

def compute_distance(route, cities):
    # Compute the distance of a route
    distance = 0
    for i in range(len(route)):
        j = (i + 1) % len(route)
        city1, city2 = cities[route[i]], cities[route[j]]
        distance += math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)
    return distance

# Example usage
cities = [(0, 0), (1, 2), (3, 1), (2, 3)]
route, distance = tsp_sa(cities)
print("Route: ", route)
print("Distance: ", distance)


Route:  [3, 2, 0, 1]
Distance:  9.048627177541055


## Genetic Algorithm:

In [6]:
import random
import math

def tsp_ga(cities, pop_size=50, elite_size=10, mutation_rate=0.01, generations=500):
    # Create initial population
    population = [create_route(cities) for i in range(pop_size)]

    # Evolution loop
    for i in range(generations):
        # Rank the population by fitness
        fitness_scores = [(route_fitness(route), route) for route in population]
        fitness_scores.sort()

        # Select the elite routes
        elite = [fitness_scores[i][1] for i in range(elite_size)]

        # Generate new population through crossover and mutation
        new_population = elite[:]
        while len(new_population) < pop_size:
            # Select parents and perform crossover
            parent1, parent2 = random.sample(elite, 2)
            child = crossover(parent1, parent2)

            # Apply mutation
            mutate(child, mutation_rate)

            # Add the child to the new population
            new_population.append(child)

        # Set the new population as the current population
        population = new_population

    # Rank the final population by fitness and return the best route
    fitness_scores = [(route_fitness(route), route) for route in population]
    fitness_scores.sort()
    return fitness_scores[0][1], fitness_scores[0][0]

def create_route(cities):
    # Create a random route that visits each city exactly once
    route = list(range(len(cities)))
    random.shuffle(route)
    return route

def crossover(parent1, parent2):
    # Perform ordered crossover to create a new child route
    n = len(parent1)
    a = random.randint(0, n-1)
    b = random.randint(a, n-1)
    child = [-1] * n

    # Copy the selected segment from parent 1 to the child
    for i in range(a, b+1):
        child[i] = parent1[i]

    # Copy the remaining cities from parent 2 to the child
    j = 0
    for i in range(n):
        if j == a:
            j = b+1
        if parent2[i] not in child:
            child[j] = parent2[i]
            j += 1

    return child

def mutate(route, mutation_rate):
    # Swap two cities with a small probability
    for i in range(len(route)):
        if random.random() < mutation_rate:
            j = random.randint(0, len(route)-1)
            route[i], route[j] = route[j], route[i]

def route_fitness(route):
    # Compute the total distance of a route
    distance = 0
    for i in range(len(route)):
        distance += get_distance(cities[route[i]], cities[route[(i+1) % len(route)]])
    return 1 / distance

def get_distance(city1, city2):
    # Compute the Euclidean distance between two cities
    return math.sqrt((city1[0] - city2[0])**2 + (city1[1] - city2[1])**2)

# Example usage
cities = [(0, 0), (1, 2), (3, 1), (2, 3)]
route, distance = tsp_ga(cities)
print("Route: ", route)
print("Distance: ", distance)


Route:  [1, 2, 0, 3]
Distance:  0.09598669570179445


The code provided above is an **implementation of a genetic algorithm** to solve the Travelling Salesman Problem (TSP). Here is an overview of how the algorithm works and what each part of the code does:

1. **Initialization:**
    We start by generating a population of candidate solutions. Each solution is represented as an array of integers, where each integer represents a city that the salesman visits. The order of the integers in the array represents the order in which the cities are visited.
    The population size is set to 50, which means that we generate 50 candidate solutions in the beginning.
    
    
2. **Evaluation:**
    Once we have a population of candidate solutions, we need to evaluate how good each solution is. In the TSP, the goal is to minimize the total distance traveled by the salesman.
    To calculate the fitness of each candidate solution, we use the distance function, which takes an array of cities (in the order visited by the salesman) and returns the total distance traveled. The fitness of each solution is set to be the reciprocal of the distance, which means that better solutions have a higher fitness.
    The fitness values are then normalized so that they sum up to 1. This allows us to use the fitness values as probabilities in the selection step.
    
    
3. **Selection:**
    The next step is to select the parents for the next generation of candidate solutions. We use a roulette wheel selection method, where each solution is selected with a probability proportional to its fitness value.
    We randomly select two parents from the population using this method.
    
    
4. **Crossover:**
    Once we have selected the parents, we need to create offspring for the next generation. We use a crossover operator to create the offspring.
    The crossover point is chosen randomly, and the two parents exchange the cities that come after the crossover point to create two offspring.
    We repeat this step until we have generated 48 offspring.
    
    
5. **Mutation:**
    To add diversity to the population, we apply a mutation operator to some of the offspring. The mutation operator randomly swaps two cities in the offspring.
    We apply this mutation operator to 5% of the offspring.


6. **Elitism:**
    We want to preserve the best solution from the previous generation in the next generation, so we use elitism to do this. We copy the best solution from the previous generation to the new generation.


7. **Replacement:**
    Finally, we replace the old population with the new population, which consists of the offspring and the elite solution from the previous generation.
    
    
8. **Repeat:**
    We repeat this process for a fixed number of generations (in this case, 200).


After the 200 generations are complete, the best solution is returned, which represents the shortest tour found by the algorithm.

## PSO (Particle Swarm Optimization)

It is possible to use Particle Swarm Optimization (PSO) to solve the Travelling Salesman Problem (TSP). 
    Here is an overview of how PSO works for TSP:


**1 Initialization:**

    We start by generating a population of candidate solutions. Each solution is represented as an array of integers, where each integer represents a city that the salesman visits. The order of the integers in the array represents the order in which the cities are visited.
    We also initialize a swarm of particles, each of which has a position and velocity that represents a candidate solution to the TSP.


**2. Evaluation:**

    Once we have a population of candidate solutions (in the form of particles), we need to evaluate how good each solution is. In the TSP, the goal is to minimize the total distance traveled by the salesman.
    To calculate the fitness of each particle, we use the distance function, which takes an array of cities (in the order visited by the salesman) and returns the total distance traveled. 
    The fitness of each particle is set to be the reciprocal of the distance, which means that better particles have a higher fitness.


**3. Velocity Update:**

    Each particle updates its velocity using the following formula:
   **v_i = w*v_i + c_1*r_1*(p_i-x_i) + c_2*r_2*(p_g-x_i)**
    
    where v_i is the velocity of particle i, 
    w is the inertia weight, c_1 and c_2 are the cognitive and social learning factors, 
    r_1 and r_2 are random numbers between 0 and 1, 
    p_i is the best solution found by particle i so far, 
    x_i is the current position of particle i, and 
    p_g is the best solution found by the entire swarm so far.
This update formula incorporates the particle's own best position (p_i) and the best position found by the swarm (p_g) to guide the particle's movement towards better solutions.


**4. Position Update:**

    Each particle updates its position using the following formula:
   **x_i = x_i + v_i**
   
    where x_i is the current position of particle i, and 
    v_i is the velocity of particle i.
    This update formula moves the particle towards its updated velocity.


**5. Local Best Update:**

    Each particle updates its own best position (p_i) if its fitness is better than its current best position.


**6. Global Best Update:**

    The swarm updates the global best position (p_g) if any particle finds a better solution than the current global best position.


**7. Repeat:**

    We repeat the velocity update, position update, and best updates for a fixed number of iterations (in this case, 200).
After the iterations are complete, the best solution is returned, which represents the shortest tour found by the algorithm.

In [7]:
import random
import math

# Set the problem parameters
c1 = 2
c2 = 2
w = 0.7
num_particles = 10
max_iterations = 100
search_space = (-10, 10)
dimensions = 2
target_error = 1e-6

class Particle:
    def __init__(self):
        self.position = [random.uniform(search_space[0], search_space[1]) for i in range(dimensions)]
        self.velocity = [0 for i in range(dimensions)]
        self.best_position = self.position
        self.error = float('inf')

    def update_velocity(self, global_best_position):
        for i in range(dimensions):
            r1 = random.random()
            r2 = random.random()

            cognitive_component = c1 * r1 * (self.best_position[i] - self.position[i])
            social_component = c2 * r2 * (global_best_position[i] - self.position[i])

            self.velocity[i] = w * self.velocity[i] + cognitive_component + social_component

    def update_position(self):
        for i in range(dimensions):
            self.position[i] += self.velocity[i]
            if self.position[i] < search_space[0]:
                self.position[i] = search_space[0]
            elif self.position[i] > search_space[1]:
                self.position[i] = search_space[1]

    def evaluate(self, cost_function):
        self.error = cost_function(self.position)
        if self.error < cost_function(self.best_position):
            self.best_position = self.position

class PSO:
    def __init__(self, cost_function):
        self.cost_function = cost_function
        self.global_best_position = [0 for i in range(dimensions)]
        self.global_best_error = float('inf')
        self.swarm = [Particle() for i in range(num_particles)]

    def optimize(self):
        for i in range(max_iterations):
            for particle in self.swarm:
                particle.evaluate(self.cost_function)
                if particle.error < self.global_best_error:
                    self.global_best_error = particle.error
                    self.global_best_position = particle.position

            for particle in self.swarm:
                particle.update_velocity(self.global_best_position)
                particle.update_position()

            if abs(self.global_best_error - target_error) < target_error:
                break

        return self.global_best_position

def sphere_function(x):
    return sum([xi**2 for xi in x])

pso = PSO(sphere_function)
print(pso.optimize())


[2.449899525416755, -7.6228180911157075]


In this example, we set the problem parameters such as the cognitive and social acceleration coefficients (c1 and c2), the inertia weight (w), the number of particles (num_particles), the maximum number of iterations (max_iterations), the search space (search_space), the number of dimensions (dimensions), and the target error (target_error).

We define the Particle class to represent a particle in the swarm. It has attributes for the position, velocity, best position, and error. It also has methods to update the velocity and position, as well as to evaluate the error using the cost function.

We define the PSO class to represent the entire swarm. It has attributes for the cost function and the global best position and error. It also has a optimize method that performs the PSO algorithm.

In the optimize method, we first evaluate the error for each particle and update the global best position and error if necessary. Then we update the velocity and position of each particle. Finally, we check if the global best error has reached

In [8]:
import numpy as np
import math
import random

def tsp_pso(points, num_particles, max_iter):
    # Initialize particles and velocities
    dim = len(points)
    max_val = dim - 1
    min_val = 0
    swarm_pos = np.zeros((num_particles, dim), dtype=int)
    swarm_vel = np.zeros((num_particles, dim), dtype=int)
    swarm_best_pos = np.zeros((num_particles, dim), dtype=int)
    swarm_best_dist = np.zeros(num_particles)
    global_best_pos = np.zeros(dim, dtype=int)
    global_best_dist = np.inf
    
    for i in range(num_particles):
        swarm_pos[i] = random.sample(range(dim), dim)
        swarm_best_pos[i] = np.copy(swarm_pos[i])
        swarm_best_dist[i] = tsp_dist(points, swarm_best_pos[i])
        if swarm_best_dist[i] < global_best_dist:
            global_best_pos = np.copy(swarm_best_pos[i])
            global_best_dist = swarm_best_dist[i]
            
    # Set PSO parameters
    w = 0.729    # inertia weight
    c1 = 1.49445 # cognitive weight
    c2 = 1.49445 # social weight
    
    # Run PSO
    for t in range(max_iter):
        for i in range(num_particles):
            # Update velocity
            swarm_vel[i] = w * swarm_vel[i] \
                + c1 * random.uniform(0, 1) * (swarm_best_pos[i] - swarm_pos[i]) \
                + c2 * random.uniform(0, 1) * (global_best_pos - swarm_pos[i])
            # Update position
            swarm_pos[i] = np.remainder(swarm_pos[i] + swarm_vel[i], dim)
            # Update personal best position
            if tsp_dist(points, swarm_pos[i]) < swarm_best_dist[i]:
                swarm_best_pos[i] = np.copy(swarm_pos[i])
                swarm_best_dist[i] = tsp_dist(points, swarm_best_pos[i])
                # Update global best position
                if swarm_best_dist[i] < global_best_dist:
                    global_best_pos = np.copy(swarm_best_pos[i])
                    global_best_dist = swarm_best_dist[i]
                    
    return global_best_pos, global_best_dist

def tsp_dist(points, tour):
    d = 0
    for i in range(len(tour)-1):
        d += math.sqrt((points[tour[i]][0]-points[tour[i+1]][0])**2 + \
                       (points[tour[i]][1]-points[tour[i+1]][1])**2)
    d += math.sqrt((points[tour[-1]][0]-points[tour[0]][0])**2 + \
                   (points[tour[-1]][1]-points[tour[0]][1])**2)
    return d

# Example usage
points = np.array([(2,2), (2,8), (7,5), (5,2), (8,3), (7,8)])
num_particles = 50
max_iter = 100
best_tour, best_dist = tsp_pso(points, num_particles, max_iter)
print('Best tour:', best_tour)
print('Best distance:', best_dist)


Best tour: [2 4 4 2 2 2]
Best distance: 4.47213595499958


In [10]:
# Example usage
cities = np.array([(0, 0), (1, 2), (3, 1), (2, 3)])
num_particles = 50
max_iter = 100
best_tour, best_dist = tsp_pso(cities, num_particles, max_iter)
print('Best tour:', best_tour)
print('Best distance:', best_dist)

Best tour: [1 1 1 1]
Best distance: 0.0


In [ ]:
import random
import math

# Define the objective function
def distance(point1, point2):
    return math.sqrt((point1[0]-point2[0])**2 + (point1[1]-point2[1])**2)

def objective_function(route):
    total_distance = 0
    for i in range(len(route)-1):
        total_distance += distance(route[i], route[i+1])
    return total_distance

# Define the PSO algorithm
class Particle:
    def __init__(self, bounds):
        self.position = [random.uniform(bounds[i][0], bounds[i][1]) for i in range(len(bounds))]
        self.velocity = [0.0 for i in range(len(bounds))]
        self.best_position = self.position.copy()
        self.best_cost = float('inf')

    def update_velocity(self, global_best_position, w, c1, c2):
        for i in range(len(self.position)):
            r1 = random.random()
            r2 = random.random()
            social = c1 * r1 * (global_best_position[i] - self.position[i])
            cognitive = c2 * r2 * (self.best_position[i] - self.position[i])
            self.velocity[i] = w * self.velocity[i] + social + cognitive

    def update_position(self, bounds):
        for i in range(len(self.position)):
            self.position[i] += self.velocity[i]
            if self.position[i] < bounds[i][0]:
                self.position[i] = bounds[i][0]
            elif self.position[i] > bounds[i][1]:
                self.position[i] = bounds[i][1]

    def evaluate(self, objective_function):
        cost = objective_function(self.position)
        if cost < self.best_cost:
            self.best_position = self.position.copy()
            self.best_cost = cost

def particle_swarm_optimization(objective_function, bounds, n_particles, n_iterations, w, c1, c2):
    particles = [Particle(bounds) for i in range(n_particles)]
    best_global_position = [0.0 for i in range(len(bounds))]
    best_global_cost = float('inf')
    for i in range(n_iterations):
        for particle in particles:
            particle.update_velocity(best_global_position, w, c1, c2)
            particle.update_position(bounds)
            particle.evaluate(objective_function)
            if particle.best_cost < best_global_cost:
                best_global_position = particle.best_position.copy()
                best_global_cost = particle.best_cost
        print(f"Iteration {i+1}, Best Cost: {best_global_cost}")
    return (best_global_position, best_global_cost)

# Example usage
cities = [(2, 5), (5, 8), (7, 3), (1, 1), (6, 9)]
bounds = [(0, 10), (0, 10)]
n_particles = 50
n_iterations = 100
w = 0.5
c1 = 1.5
c2 = 1.5

best_route, best_cost = particle_swarm_optimization(objective_function, bounds, n_particles, n_iterations, w, c1, c2)
print(f"Best Route: {best_route}, Best Cost: {best_cost}")


In [ ]:
import numpy as np
import math
import random

# define the problem
cities = np.array([(60, 200), (180, 200), (80, 180), (140, 180), (20, 160), (100, 160), (200, 160), (140, 140), (40, 120), (100, 120), (180, 100), (60, 80), (120, 80), (180, 60), (20, 40), (100, 40), (200, 40), (20, 20), (60, 20), (160, 20)])

num_particles = 20
num_dimensions = cities.shape[0]
max_iter = 1000
w = 0.729844  # inertia weight
c1 = 1.496180  # cognitive weight
c2 = 1.496180  # social weight

# def calculate_path_length(path):
#     """
#     Calculate the total length of a path through all cities
#     """
#     total_distance = 0
#     for i in range(num_dimensions-1):
#         total_distance += math.sqrt((cities[path[i], 0] - cities[path[i+1], 0]) ** 2 + (cities[path[i], 1] - cities[path[i+1], 1]) ** 2)
#     total_distance += math.sqrt((cities[path[num_dimensions-1], 0] - cities[path[0], 0]) ** 2 + (cities[path[num_dimensions-1], 1] - cities[path[0], 1]) ** 2)
#     return total_distance

def calculate_path_length(path):
    """
    Calculate the total length of a path through all cities
    """
    total_distance = 0
    for i in range(num_dimensions-1):
        city1 = cities[path[i]]
        city2 = cities[path[i+1]]
        total_distance += math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)
    city1 = cities[path[num_dimensions-1]]
    city2 = cities[path[0]]
    total_distance += math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)
    return total_distance


def PSO():
    global num_particles, num_dimensions, max_iter, w, c1, c2

    # initialize the swarm
    swarm_pos = np.zeros((num_particles, num_dimensions))
    swarm_vel = np.zeros((num_particles, num_dimensions))
    swarm_best_pos = np.zeros((num_particles, num_dimensions))
    swarm_best_distance = np.ones(num_particles) * np.inf
    global_best_pos = np.zeros(num_dimensions)
    global_best_distance = np.inf

    for i in range(num_particles):
        swarm_pos[i] = np.random.permutation(num_dimensions)
        swarm_best_pos[i] = swarm_pos[i].copy()
        swarm_best_distance[i] = calculate_path_length(swarm_best_pos[i])

        if swarm_best_distance[i] < global_best_distance:
            global_best_pos = swarm_best_pos[i].copy()
            global_best_distance = swarm_best_distance[i]

    # update the swarm
    for i in range(max_iter):
        for j in range(num_particles):
            # update velocity
            swarm_vel[j] = w * swarm_vel[j] + c1 * random.random() * (swarm_best_pos[j] - swarm_pos[j]) + c2 * random.random() * (global_best_pos - swarm_pos[j])

            # update position
            swarm_pos[j] = np.roll(swarm_pos[j] + swarm_vel[j], random.randint(0,num_dimensions-1))

            # calculate the distance
            distance = calculate_path_length(swarm_pos[j])

            # update local best position
            if distance < swarm_best_distance[j]:
                swarm_best_pos[j] = swarm_pos[j].copy()
                swarm_best_distance[j] = distance

                # update global best position
                if swarm_best_distance[j] < global_best_distance:
                    global_best_pos = swarm_best_pos[j].copy()
                    global_best_distance = swarm_best_distance[j]

                print("Iteration:", i, " Best Distance: ", global_best_distance)

    # return the best path and its length
    return global_best_pos, global_best_distance

# run the PSO algorithm
best_path, best_distance = PSO()

print("Best path found:", best_path)
print("Length of best path:", best_distance)
